In [1]:
import numpy as np
import tensorflow as tf
from __future__ import division, print_function, unicode_literals

# to make this notebook's output stable across runs
def reset_graph(seed=42):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)
    

from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense, Input
from keras.datasets import mnist
from keras import optimizers
from keras import utils
from keras.models import Model
from keras.layers.normalization import BatchNormalization
from keras.layers import Lambda
from keras.backend import shape
from keras import backend as K


from keras.layers import Lambda
from keras.engine.training import _slice_arrays

print("done")

Using Theano backend.


done


In [2]:
# TODO: Replace <FILL IN> with appropriate code

# to reset the Tensorflow default graph
reset_graph()

# Building the AlexNet model

model = Sequential()
#model.add(inputs = Input(shape=(224, 224, 3)))
model.add(Conv2D(name='conv1', filters=96, input_shape=(224,224,3), kernel_size=(11,11),
                 strides=(4,4), padding='same'))

model.add(MaxPooling2D(pool_size=(3, 3), strides=2, padding='valid'))
model.add(Conv2D(filters = 256, kernel_size = 5,strides = 1,
                 activation = 'relu', padding='same', name='conv2'))
model.add(MaxPooling2D(pool_size=(3, 3), strides=2, padding='valid', data_format=None))
#model.add(BatchNormalization())

model.add(Conv2D(filters = 384, kernel_size = 3,strides = 1,activation = 'relu', padding='same', name='conv3'))
#model.add(BatchNormalization())
model.add(Conv2D(filters = 384, kernel_size = 3,strides = 1, activation = 'relu', padding='same', name='conv4'))
#model.add(BatchNormalization())
model.add(Conv2D(filters = 256, kernel_size = 3,strides = 1,activation = 'relu', padding='same', name='conv5'))
#model.add(BatchNormalization())

#Flatten
model.add(Flatten())
model.add(Dense(units = 4096, activation = 'relu', name='fc6'))
#model.add(BatchNormalization())
model.add(Dense(units = 4096, activation = 'relu', name='fc7'))
#model.add(BatchNormalization())
model.add(Dense(units = 1000, activation = 'softmax', name='fc8'))
# PRINT OUT THE MODEL
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

### LOAD IN THE WEIGHTS
weights_dict = np.load('bvlc_alexnet.npy', encoding='bytes').item()       
names = [weight.name for layer in model.layers for weight in layer.weights]
weights = model.get_weights()

myAppend = []

for name, weight in zip(names, weights):
    name = name.split('/')[0]
    # Note: Here, the name of the weights appear two times in the file. 
    # My initial thoughts were that they were the separate channels, but after looking at them
    # they are the same, both size 48 on axis=3
    if name in weights_dict and name not in myAppend:
        # Add them to temporary array so I'm not setting the weights two times
        myAppend.append(name)
        print(name)
        # Prints out the loaded dimensions
        print('load  weight shape',weights_dict[name][0].shape,'bias', weights_dict[name][1].shape)
        # Prints out the layer shape of the keras model I constructed
        print('model weight shape',model.get_layer(name).get_weights()[0].shape,'bias',model.get_layer(name).get_weights()[1].shape)
        # So here is where I do the first trick....
        # Basicly, conv2,4 and 5 are split into two channels just as the paper describes
        # In my case, the conv2 is of dimension 48 when my model is 96 (96/2 = 48)
        # The same thing happens with conv4 and 5, so to be able to load in the weights
        if(name == 'conv2' or name == 'conv4' or name == 'conv5'):
            # I create two versions of them and stack the together on axis=3
            a = weights_dict[name][0]
            b = weights_dict[name][0]
            c = np.concatenate((a,b), axis= 2)
            # This will create an array of weights of size 5,5,96, just as my model is
            model.get_layer(name).set_weights([c,weights_dict[name][1]])
        # NOTE: This is where I might need help, I'm unsure if my model is creating too many
        # parameters when I flatten the conv5, but you can see that the loaded weights are
        # (9216, 4096) bias (4096,)
        # While my model is 
        # (43264, 4096) bias (4096,)
        # I need some help regarding this, so I also wanted to ask you about this.
        elif (name != 'fc6'):
            model.get_layer(name).set_weights(weights_dict[name])
       


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1 (Conv2D)               (None, 56, 56, 96)        34944     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 27, 27, 96)        0         
_________________________________________________________________
conv2 (Conv2D)               (None, 27, 27, 256)       614656    
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 13, 13, 256)       0         
_________________________________________________________________
conv3 (Conv2D)               (None, 13, 13, 384)       885120    
_________________________________________________________________
conv4 (Conv2D)               (None, 13, 13, 384)       1327488   
_________________________________________________________________
conv5 (Conv2D)               (None, 13, 13, 256)       884992    
__________

In [3]:
# TODO: Replace <FILL IN> with appropriate code
# test the AlexNet model on the given images

import cv2
import os

#get list of all images
current_dir = os.getcwd()
image_path = os.path.join(current_dir, 'test_images')
img_files = [os.path.join(image_path, f) for f in os.listdir(image_path) if f.endswith('.jpg')]
# get all labels
lines = [line.rstrip('\n') for line in open('labels.txt')]

#load all images
imgs = []
for f in img_files:
    print("reading",f)
    imgs.append(cv2.imread(f))

# Loop over all images
allimgs = np.zeros(shape=(3,224,224,3), dtype=np.float)

reading C:\Users\ottarg\Desktop\KTH_PROGRAM\6-Scalable Machine Learning and Deep Learning\lab2\test_images\test_image1.jpg
reading C:\Users\ottarg\Desktop\KTH_PROGRAM\6-Scalable Machine Learning and Deep Learning\lab2\test_images\test_image2.jpg
reading C:\Users\ottarg\Desktop\KTH_PROGRAM\6-Scalable Machine Learning and Deep Learning\lab2\test_images\test_image3.jpg


In [4]:

for i, image in enumerate(imgs):
    img = cv2.resize(image.astype(np.float32), (224, 224))
    
    # **** FROM PAPER ****
    #We did not pre-process the images
    #in any other way, except for subtracting the mean activity over the training set from each pixel. So
    #we trained our network on the (centered) raw RGB values of the pixels.
    # Note: This should only be for the training I think
    #imagenet_mean = np.array([mean.r., mean.g., mean.b.], dtype=np.float32)
    #img -= imagenet_mean

    allimgs[i] = img.reshape(1,224,224,3)
    print("IMG")
    #print(img)
    print("test")
    predict = model.predict(img.reshape(1,224,224,3))
    predictVal = np.max(predict)
    predictIndex = np.argmax(predict)
    print("predict")
    print("shape",predict.shape)
    #print(predict)
    print(predictVal)
    print(predictIndex)
    print(lines[predictIndex])
    print()

IMG
test
predict
shape (1, 1000)
0.00148037
788
788: 'shoe shop, shoe-shop, shoe store',

IMG
test
predict
shape (1, 1000)
0.00148037
788
788: 'shoe shop, shoe-shop, shoe store',

IMG
test
predict
shape (1, 1000)
0.00145519
788
788: 'shoe shop, shoe-shop, shoe store',

